# 经典案例 波士顿房价预测

## 数据集介绍

该数据集包含美国人口普查局收集的美国马萨诸塞州波士顿住房价格的有关信息, 数据集很小，只有506个案例。
数据集都有以下14个属性:

|属性|数据类型|字段描述|
|-----|-----|-----|
| CRIM |Float  |城镇人均犯罪率|
| ZN | Float  |占地面积超过25,000平方英尺的住宅用地比例 |
| INDUS | Float  |每个城镇非零售业务的比例 |
| CHAS | Integer  |Charles River虚拟变量（如果是河道，则为1;否则为0 |
| NOX | Float  |一氧化氮浓度（每千万份）|
| RM | Float  |每间住宅的平均房间数   |
| AGE | Float  |1940年以前建造的自住单位比例|
| DIS |Float  |波士顿的五个就业中心加权距离|
| RAD | Integer  |径向高速公路的可达性指数|
| TAX | Float  |每10,000美元的全额物业税率|
| PTRATIO | Float  |城镇的学生与教师比例|
| B | Float  | 1000（Bk - 0.63）^ 2 其中Bk是城镇黑人的比例 |
| LSTAT | Float  |人口状况下降％ |
| MEDV | Float  |自有住房的中位数报价, 单位1000美元 |

### 第 1 步，数据加载与清洗

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

plt.style.use('ggplot')
# 读入数据 Read-In Data将housing,csv读入
data = pd.read_csv('../../data/boston_house_prices.csv') # 读入数据
data.info() # 数据一共有14个特征，并且没有缺失值

In [ ]:
from sklearn.datasets import load_boston
dataset = load_boston()  # 直接从sklearn中获取波士顿房屋数据集
print(dataset.DESCR)
X = load_boston().data
y = load_boston().target
df = pd.DataFrame(X, columns=dataset.feature_names)
df.head()

In [ ]:
# 数据特征
# 1 查看 相关性检验
plt.figure(figsize=(12,8))
sns.heatmap(data.corr(), annot=True, fmt='.2f', cmap='PuBu')
data.corr()['MEDV'].sort_values() # 查询和MEDV字段的相关性

In [ ]:
# 多变量研究
sns.pairplot(data[["LSTAT","RM","PIRATIO","MEDV"]]) #

### 第 2 步，创建数据集

In [ ]:
# 划分训练集和测试集
X ,y = data[data.columns.delete(-1)], data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=888)

### 第 3 步，建立模型

In [ ]:
linear_model = LinearRegression()  # 建立线性回归模型
linear_model.fit(X_train, y_train)
coef = linear_model.coef_  #回归系数
line_pre = linear_model.predict(X_test)
print('SCORE:{:.4f}'.format(linear_model.score(X_test, y_test)))
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, line_pre))))
# 根据结果来看，预测的score为76%左右，然后均方误差RMSE大约是4.5，为了更好的看出预测数据的问题，试着可视化一下


In [ ]:
df_coef = pd.DataFrame()
df_coef['Title'] = data.columns.delete(-1)
df_coef['Coef'] = coef # 线性回归的相关系数

hos_pre = pd.DataFrame()
hos_pre['Predict'] = line_pre
hos_pre['Truth'] = y_test.values
hos_pre.plot(figsize=(18,8))
# • 得出score为76%，从上图得知预测的房价整体偏小，在10-35这个区间预测结果较为准确，超过这个区间预测价格偏小，可进一步对原始数据离群值做处理。
# • 数据比较干净，练习起来缺少特征工程部分的工作，而且是预测性模型，相比分类问题少了模型选择上的空间。

### 第 4 步，评价模型

In [ ]:
plt.scatter(y_test, line_pre,label='y')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4,label='predicted')

# 然后在整个数据集中评价模型
line_pre_all = linear_model.predict(X)  #预测值
print('SCORE:{:.4f}'.format(linear_model.score(X,y)))
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y, line_pre_all))))

In [ ]:
hos_pre_all = pd.DataFrame()
hos_pre_all['Predict'] = line_pre_all
hos_pre_all['Truth'] = y
hos_pre_all.plot(figsize=(18,8))
plt.scatter(y, line_pre_all,label='y')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4,label='predicted') # 整个数据集结果的可视化

### 第 5 步，优化模型

由以上分析可知，模型在整个数据集中的评分比在测试集中要低
进一步探索和模型改进
• 尝试使用相关性最高的3个特征量重建模型，并与原模型进行比较
• 尝试使用其它多种算法分别建立模型，并比较模型

In [ ]:
# 首先我尝试相关性最高的三个特征重建模型，去与原模型比较一下
data.corr()['MEDV'].abs().sort_values(ascending=False).head(4)

In [ ]:
# 由此我们得出了三个相关性最高的特征，我们将其作为自变量去建立模型
X2 = np.array(data[['LSTAT','RM','PIRATIO']])
X2_train, X2_test, y_train, y_test = train_test_split(X2, y, random_state=1,test_size=0.2)
linear_model2 = LinearRegression()
linear_model2.fit(X2_train,y_train)
print(linear_model2.intercept_)
print(linear_model2.coef_)

In [ ]:
line2_pre = linear_model2.predict(X2_test)  #预测值
print('SCORE:{:.4f}'.format(linear_model2.score(X2_test, y_test)))          # 模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, line2_pre)))) # RMSE(标准误差)

# 我们可以得到，对于预测测试集的数据的得分score明显是没有开始的线性回归模型1高的，然后我们再看看，在整个数据集中它的表现
line2_pre_all = linear_model2.predict(X2)  #预测值
print('SCORE:{:.4f}'.format(linear_model2.score(X2, y)))#模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y, line2_pre_all))))#RMSE(标准误差)

这样比较下来，第一个模型达到的分数，即使在整个数据集中73%，但是这个模型的得分大约是67.6%，
由此可以得出，第一个模型还是比这个模型优的，接下来就需要尝试更多的模型了

数据标准化
数据集的 标准化 对scikit-learn中实现的大多数机器学习算法来说是 常见的要求 。
如果个别特征或多或少看起来不是很像标准正态分布(具有零均值和单位方差)，那么它们的表现力可能会较差。
所以我这里首先对数据进行了一个标准化处理

In [ ]:
from sklearn.datasets import load_boston
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
X_train, X_test, y_train, y_test = train_test_split( data, target, test_size=0.2, random_state=888)

from sklearn.preprocessing import StandardScaler
ss_x = StandardScaler()
X_train = ss_x.fit_transform(X_train)
X_test = ss_x.transform(X_test)
ss_y = StandardScaler()
y_train = ss_y.fit_transform(y_train.reshape(-1, 1))
y_test = ss_y.transform(y_test.reshape(-1, 1))

### GradientBoosting（梯度提升）

In [ ]:
from sklearn import ensemble
# params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 1,'learning_rate': 0.01, 'loss': 'ls'}
# clf = ensemble.GradientBoostingRegressor(**params)
clf = ensemble.GradientBoostingRegressor()
clf.fit(X_train, y_train)
clf_pre=clf.predict(X_test) #预测值
print('SCORE:{:.4f}'.format(clf.score(X_test, y_test))) # 模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, clf_pre)))) # RMSE(标准误差)

###  Lasso回归 （Least AbsoluteShrinkage and SelectionOperator）
Lasso 也是修正回归系数的绝对值。
与岭回归不同的是，Lasso回归在惩罚方程中用的是绝对值，而不是平方。这就使得惩罚后的值可能会变成0

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score


lasso = Lasso()
lasso.fit(X_train, y_train)
y_predict_lasso = lasso.predict(X_test)
r2_score_lasso = r2_score(y_test, y_predict_lasso)
print('SCORE:{:.4f}'.format(lasso.score(X_test, y_test)))  #模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, y_predict_lasso))))  #RMSE(标准误差)
print('Lasso模型的R-squared值为:', r2_score_lasso)

###  ElasticNet
ElasticNet回归是Lasso回归和岭回归的组合

In [ ]:
from sklearn.linear_model import ElasticNet

enet = ElasticNet()
enet.fit(X_train, y_train)
y_predict_enet = enet.predict(X_test)
r2_score_enet = r2_score(y_test, y_predict_enet)
print('SCORE:{:.4f}'.format(enet.score(X_test, y_test)))  #模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, y_predict_enet))))  #RMSE(标准误差)
print("ElasticNet模型的R-squared值为:", r2_score_enet)

### Support Vector Regression(SVR)

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import r2_score as r2, mean_squared_error as mse, mean_absolute_error as mae

def svr_model(kernel):
    svr = SVR(kernel=kernel)
    svr.fit(X_train, y_train)
    y_predict = svr.predict(X_test)
    # score(): Returns the coefficient of determination R^2 of the prediction.
    print(kernel, ' SVR的默认衡量评估值值为：', svr.score(X_test, y_test))
    print(kernel, ' SVR的R-squared值为：', r2(y_test, y_predict))
    print(kernel, ' SVR的均方误差（mean squared error）为：', mse(y_test, y_predict))
    print(kernel, ' SVR的平均绝对误差（mean absolute error）为：', mae(y_test, y_predict))
    # print(kernel,' SVR的均方误差（mean squared error）为：',mse(scalery.inverse_transform(y_test), scalery.inverse_transform(y_predict)))
    # print(kernel,' SVR的平均绝对误差（mean absolute error）为：',mae(scalery.inverse_transform(y_test),scalery.inverse_transform(y_predict)))
    return svr


# linear 线性核函数
# linear_svr = svr_model(kernel='linear')
# poly 多项式核
# poly_svr = svr_model(kernel='poly')
# rbf（Radial Basis  Function） 径向基函数
# rbf_svr = svr_model(kernel='rbf')

### SVM（支持向量机）回归 - - 线性核

In [ ]:
from sklearn.svm import SVR

linear_svr = SVR(kernel="linear")
linear_svr.fit(X_train, y_train)
linear_svr_pre = linear_svr.predict(X_test)  #预测值
print('SCORE:{:.4f}'.format(linear_svr.score(X_test, y_test)))  #模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, linear_svr_pre))))  #RMSE(标准误差)

## SVM（支持向量机）回归 - - 多项式核
在使用SVM回归 - - 多项式核的时候，首先要对数据进行一个标准化处理

In [ ]:
from sklearn.preprocessing import StandardScaler
ss_x = StandardScaler()
X_train = ss_x.fit_transform(X_train)
X_test = ss_x.transform(X_test)
ss_y = StandardScaler()
y_train = ss_y.fit_transform(y_train.reshape(-1, 1))
y_test = ss_y.transform(y_test.reshape(-1, 1))

# 再进行建立模型来预测
poly_svr = SVR(kernel="poly")
poly_svr.fit(X_train, y_train)
poly_svr_pre = poly_svr.predict(X_test)  #预测值
print('SCORE:{:.4f}'.format(poly_svr.score(X_test, y_test)))  #模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, poly_svr_pre))))  #RMSE(标准误差)

### 决策树回归

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X_train, y_train)
tree_reg_pre = tree_reg.predict(X_test)  #预测值
print('SCORE:{:.4f}'.format(tree_reg.score(X_test, y_test)))  #模型评分
print('RMSE:{:.4f}'.format(np.sqrt(mean_squared_error(y_test, tree_reg_pre))))  #RMSE(标准误差)

